In [1]:
import numpy as np
import math
import pickle
import csv

In [38]:
# 106 columns

X_train = []
Y_train = []

with open('data/X_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        X_train.append([float(i) for i in row])
X_train = np.array(X_train)

with open('data/Y_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        Y_train.append(float(row[0]))
Y_train = np.array(Y_train)

X_0 = np.copy(X_train[np.where(Y_train==0)])
X_1 = np.copy(X_train[np.where(Y_train==1)])

X_0_cont = X_0[:,[0,1,3,4,5]]
# X_0_cont = np.copy(X_train[np.where(Y_train==0)])
X_0_disc = np.delete(X_0, [0,1,3,4,5], 1)
X_1_cont = X_1[:,[0,1,3,4,5]]
# X_1_cont = np.copy(X_train[np.where(Y_train==1)])
X_1_disc = np.delete(X_1, [0,1,3,4,5], 1)

prob0 = len(X_0) / len(X_train)
prob1 = len(X_1) / len(X_train)

### accuracy is better if different covariance matrixes are used

In [39]:
# make the naive assumption that dimension is independent, which is probably wrong for both Gaussian and Bernoulli
mu_0 = np.mean(X_0_cont, axis=0)
mu_1 = np.mean(X_1_cont, axis=0)
cov_0 = np.cov(X_0_cont, rowvar=0)
cov_1 = np.cov(X_1_cont, rowvar=0)

# cov_0 = cov_0_before*prob0/(prob0+prob1) + cov_1_before*prob1/(prob0+prob1)
# cov_1 = cov_0_before*prob0/(prob0+prob1) + cov_1_before*prob1/(prob0+prob1)

# array: prob of a discrete feature being 0
prob0_0 = (X_0_disc==0).sum(axis=0) / len(X_0_disc)
prob0_1 = (X_1_disc==0).sum(axis=0) / len(X_1_disc)

In [40]:
with open('data/X_test.csv', 'rt', encoding='big5') as infile:
    reader = csv.reader(infile)
    row1 = next(reader) # skip headings
    with open('data/submission_gen.csv', 'wt') as outfile:
        test_writer = csv.writer(outfile)
        test_writer.writerow(['id','label'])
        
        count = 0
        for row in reader:
            X_input = np.array([float(i) for i in row])
            X_input_cont = X_input[[0,1,3,4,5]]
            # X_input_cont = np.array([float(i) for i in row])
            X_input_disc = np.copy(np.delete(X_input, [0,1,3,4,5]))
            
            # print('here ok 1')
            
            # np.linalg.inv
            # np.linalg.pinv
            denom_0 = pow(np.linalg.det(cov_0),0.5)
            denom_1 = pow(np.linalg.det(cov_1),0.5)
            if np.isnan(denom_0) or denom_0 == 0:
                denom_0 = pow(10,-8)
            if np.isnan(denom_1) or denom_1 == 0:
                denom_1 = pow(10,-8)
            
            X_cont_in_0 = 1 / (pow(2*math.pi, len(X_input_cont)/2) + pow(10,-8)) / denom_0 \
            * math.exp(-0.5*(X_input_cont - mu_0).T.dot(np.linalg.pinv(cov_0)).dot(X_input_cont - mu_0))
            
            X_cont_in_1 = 1 / (pow(2*math.pi, len(X_input_cont)/2) + pow(10,-8)) / denom_1 \
            * math.exp(-0.5*(X_input_cont - mu_1).T.dot(np.linalg.pinv(cov_1)).dot(X_input_cont - mu_1))
            
            # print('here ok 2')
            
            X_input_disc_0 = np.copy(np.delete(X_input, [0,1,3,4,5]))
            X_input_disc_1 = np.copy(np.delete(X_input, [0,1,3,4,5]))
            for idx in range(0, len(X_input_disc)):
                if(X_input_disc[idx] == 0):
                    X_input_disc_0[idx] = prob0_0[idx]
                    X_input_disc_1[idx] = prob0_1[idx]
                else:
                    X_input_disc_0[idx] = 1 - prob0_0[idx]
                    X_input_disc_1[idx] = 1 - prob0_1[idx]
            # multiply all the probabilities together
            X_disc_in_0 = X_input_disc_0.prod()
            X_disc_in_1 = X_input_disc_1.prod()
            
            prob_X_in_0 = X_disc_in_0 * X_cont_in_0
            prob_X_in_1 = X_disc_in_1 * X_cont_in_1
            '''prob_X_in_0 = X_cont_in_0
            prob_X_in_1 = X_cont_in_1'''
            
            if (prob_X_in_0 * prob0 + prob_X_in_1 * prob1) == 0:
                prob_0_given_x = 0.5
                print('both prob_X_in_0 and prob_X_in_1 are 0')
            else:
                prob_0_given_x = prob_X_in_0 * prob0 / (prob_X_in_0 * prob0 + prob_X_in_1 * prob1)
            
            if np.isnan(prob_0_given_x):
                print('prob', prob_0_given_x)
                # print(X_input_disc)
                print('cov_0', cov_0)
                print('det(cov_0)', np.linalg.det(cov_0))
                print('pow_0.5 cov_0', pow(np.linalg.det(cov_0),0.5))
                print('denom_0', denom_0)
                print('X_cont_in_0', X_cont_in_0)
                print('===========')
                print('cov_1', cov_1)
                print('det(cov_1)', np.linalg.det(cov_1))
                print('pow_0.5 cov_1', pow(np.linalg.det(cov_1),0.5))
                print('denom_1', denom_1)
                print('X_cont_in_1', X_cont_in_1)
                # print(np.delete(X_input, [0,1,3,4,5]))
                break
                
            prediction = 1
            if prob_0_given_x > 0.5:
                prediction = 0
            
            count += 1
            test_writer.writerow([str(count), prediction])
            
print('finished writing submission file')

finished writing submission file


In [115]:
with open('gen_vars.pickle', 'wb') as f:
    pickle.dump([prob0, prob1, mu_0, mu_1, cov_0, cov_1, prob0_0, prob0_1], f)

In [114]:
# print(prob0)
# print(prob1)
'''print(len(X_0))
print(len(X_0_cont))
print(len(X_0_disc))'''
'''print(mu_0)
print(mu_1)'''
#print(np.linalg.inv(cov_1))
'''print(prob0_0)
print(prob0_1)'''
# np.linalg.det(cov_1)
# print(X_train[0])
'''print(X_input)
print(X_input_cont)
print(X_input_disc)
print(np.delete(X_input, [0,1,3,4,5]))'''

'print(X_input)\nprint(X_input_cont)\nprint(X_input_disc)\nprint(np.delete(X_input, [0,1,3,4,5]))'

In [118]:
with open('gen_vars.pickle', 'rb') as infile:
    obj0, obj1, obj2, obj3, obj4, obj5, obj6, obj7 = pickle.load(infile)
print(obj4)

[[  1.96562881e+02  -1.17796585e+05   7.23397455e+02   1.29808287e+02
    7.99420545e+00]
 [ -1.17796585e+05   1.13384741e+10  -1.33052484e+04  -4.30057403e+05
   -2.81105754e+04]
 [  7.23397455e+02  -1.33052484e+04   9.27637325e+05  -7.90546039e+03
    2.12833620e+02]
 [  1.29808287e+02  -4.30057403e+05  -7.90546039e+03   9.65691480e+04
    7.15673181e+01]
 [  7.99420545e+00  -2.81105754e+04   2.12833620e+02   7.15673181e+01
    1.51757629e+02]]
